In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这份笔记本是在[Ivan Nardini](https://github.com/inardini)的协助下编写的。

# 从头开始使用Vertex AI实验

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fget_started_with_vertex_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程涵盖了如何开始使用Vertex AI实验，并演示了如何在生产环境中使用Vertex AI。

了解更多关于[Vertex AI实验](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments)，[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的信息。

### 目标

在本教程中，您将学习如何在使用 Vertex AI 时使用 Vertex AI Experiments 进行训练。

本教程使用以下 Google Cloud ML 服务：

- Vertex AI Experiments
- Vertex ML Metadata
- Vertex AI Training

执行的步骤包括：

- 本地（笔记本）训练
    - 创建一个实验。
    - 在实验中创建第一个运行。
    - 记录参数和指标。
    - 创建工件谱系。
    - 可视化实验结果。
    - 执行第二次运行。
    - 比较实验中的两次运行。
- 云端（Vertex AI）训练
    - 在训练脚本中：
        - 创建一个实验。
        - 记录参数和指标。
        - 创建工件谱系。
    - 创建一个 Vertex AI 训练定制作业。
    - 执行定制作业。
    - 可视化实验结果。

### 推荐

在 Google Cloud 上进行 E2E MLOps 时，以下是在实验或正式训练模型时记录数据的一些最佳实践。

#### Python 日志记录

在本地进行临时培训时，请使用 Python 的日志记录包。

#### 云日志记录

在云上进行训练时，请使用云日志记录。

#### 实验

在执行实验并比较不同实验配置的结果时，请同时使用 Vertex AI 实验和日志记录。

### 数据集

这个教程没有使用数据集。对示例数据集的引用仅用于演示目的。

成本
本教程使用Google Cloud的收费组件：
- Vertex AI
了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。请等待直到完成后再继续下一步。⚠️</b>
</div>

### 在Colab环境上进行身份验证

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并为Python初始化Vertex AI SDK

要使用Vertex AI开始，您必须拥有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶不存在: 运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION $BUCKET_URI

服务账号

**如果您不知道您的服务账号**，请尝试通过执行下面第二个单元格中的`gcloud`命令来获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
IS_COLAB = False

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        # print("shell_output=", shell_output)
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库

In [ ]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 设置硬件加速器

您可以为训练设定硬件加速器。

将变量 `TRAIN_GPU/TRAIN_NGPU` 设置为使用支持 GPU 的容器映像以及分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个 GPU 容器映像，并为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 以使用一个在 CPU 上运行的容器映像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

设置预构建的容器

设置预构建的Docker容器镜像用于训练。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.12".replace(".", "-")

if TRAIN_GPU:
    TRAIN_VERSION = "tf-gpu.{}.py310".format(TF)
else:
    TRAIN_VERSION = "tf-cpu.{}.py310".format(TF)


TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量 `TRAIN_COMPUTE` 设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存
 - `vCPUs`: 数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下机器类型不支持用于训练：*

 - `standard`: 2 vCPUs
 - `highcpu`: 2、4 和 8 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## 介绍 Vertex AI 实验

通过 Vertex AI 实验，您可以记录和跟踪以下内容，用于实验和开发模型架构和模型训练：

- 记录模型架构的元参数。
- 记录训练的超参数。
- 记录评估指标。
- 创建数据集、模型和评估的工件衍生。
- 将一个或多个训练运行分组到一个实验中。
- 比较实验结果。

Vertex AI 实验可以与以下开发流程集成：

- 在笔记本中进行本地开发
- 在 Vertex AI 训练中进行云开发
- 在 Vertex AI 管道中进行操作开发

了解更多关于[实验](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

了解更多关于[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction)。

### 在笔记本电脑中进行本地开发

您可以在本地开发中跟踪实验，例如在Vertex AI Workbench笔记本中，方法如下：

- 包装（引言）实验的创建。
- 在实验中为每个训练运行实例化一个运行。
- 在本地训练运行中，记录相应的参数和结果。
- 创建与工件和实验数据的渊源。
- 检索实验数据。

创建用于跟踪培训相关元数据的实验

首先，您可以使用 `init()` 方法创建一个实验，然后在实验中初始化一个运行，使用 `start_run()`。

- `aiplatform.init()` - 创建一个实验实例
- `aiplatform.start_run()` - 跟踪实验中的特定运行。

In [ ]:
# Specify a name for the experiment
EXPERIMENT_NAME = "[your-experiment-name]"

if EXPERIMENT_NAME == "[your-experiment-name]":
    EXPERIMENT_NAME = f"example-{uuid.uuid1()}"

In [ ]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

为实验记录参数

通常，实验与特定数据集和模型架构相关联。在一个实验中，您可能有多个训练运行，
每个运行涉及不同的配置。例如：

- 数据提供，例如：
    - 数据集拆分
    - 数据集抽样和增强
- 元参数，例如：
    - 层的深度和宽度
- 超参数，例如：
    - 批量大小
    - 学习率

这些配置设置被称为参数，您可以使用`log_params()`方法将其存储为键-值对。

In [ ]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

记录实验的指标

在实验运行结束或终止时，您可以记录结果，以便比较不同运行的结果。例如：

- 评估指标
- 超参数搜索选择
- 模型训练时间
- 提前停止触发器

这些结果被称为指标，您可以使用 `log_metrics()` 方法将其存储为键值对。

In [ ]:
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)

获取实验结果

当您完成实验中的一个运行时，请调用 `end_run()` 方法来完成该运行的日志记录。

接下来，将实验名称作为参数传递给 `get_experiment_df()` 方法，以将实验结果作为 Pandas 数据帧获取。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

开始实验中的后续运行

接下来，为同一个实验创建第二个运行。在这个示例中，您将将`units`的元参数从126更改为256，并记录不同的度量结果。

In [ ]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = 256  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = 98.8  # value changed
metrics["train_acc"] = 99.5  # value changed
aiplatform.log_metrics(metrics)

最后，使用实验名称作为方法`get_experiment_df()`的参数，以获得实验中所有运行结果的pandas数据框。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

请删除实验

然后，使用`delete()`方法来删除实验。

In [ ]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

在实验运行中创建工件血统

在这个例子中，您可以向您的实验运行中添加工件血统。首先，创建一个实验，然后在实验中开始一个运行。

In [ ]:
# Create experiment
EXPERIMENT_NAME = f"example-{uuid.uuid1()}"
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

创建数据集和模型工件

接下来，在Vertex ML Metadata中创建合成工件，以与实验中的运行相关联，作为谱系。您将创建：

- `dataset_artifact`：作为实验运行的输入的数据集。
- `model_artifact`：作为实验运行的输出的模型。

In [ ]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

#### 创建工件传承

接下来，为实验运行创建工件传承，使用`start_execution()`方法实例化一个执行。然后使用`assign_input_artifacts()`方法附加输入工件，并使用`assign_output_artifacts()`方法附加输出工件。

在这个例子中，要找到实验的传承，您可以在执行运行中添加一个合成（元数据）条目`lineage`，并将值设置为传承的控制台 URI，您可以从`get_output_artifacts()`方法和`lineage_console_uri`属性获取。

In [ ]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    aiplatform.log_params({"units": 256})
    aiplatform.log_metrics({"acc": 96.8})

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

获取实验结果

接下来，使用实验名称作为参数传递给方法`get_experiment_df()`，以获取实验结果作为一个 pandas 数据帧。

在这个示例中，您将资源 URI 存储为度量值`lineage`，在执行运行中成为血统的一部分。

In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### 可视化文物谱系

接下来，打开下面的链接来可视化文物谱系。

In [ ]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

#### 删除工件谱系

接下来，使用`delete()`方法来删除工件谱系。

In [ ]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

删除实验

接下来，您可以使用`delete()`方法来删除实验。

In [ ]:
try:
    exp.delete()
except Exception as e:
    print(e)

### 在Vertex AI Training中的云开发

您可以使用Vertex AI Training在云开发中跟踪一个实验，方法如下：

在您的Python训练脚本中，重复与本地开发相同的步骤：

- 包装（前导）实验的创建。
- 实例化每次训练运行中的一个运行。
- 在本地训练运行中，记录相应的参数和结果。
- 为艺术品和实验数据创建血统。
- 检索实验数据。

#### 包布局

在开始训练之前，您会查看Python包是如何组装用于自定义训练工作的。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是安装包到Docker镜像操作环境的指令。

文件`trainer/task.py`是执行自定义训练工作的Python脚本。*注意*，当我们在工作人员池规范中提到它时，我们将目录斜杠替换为点(`trainer.task`)并丢掉文件后缀(`.py`)。

#### 包组装

在下面的单元格中，组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'google-cloud-aiplatform',\n\n  ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Synethic Training Script for Experiments\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### 创建合成训练脚本

首先，编写一个合成训练脚本。它实际上不会训练模型，而是模拟模型的训练：

- 参数解析
  - `experiment`：实验的名称。
  - `run`：实验中的运行名称。
  - `epochs`：训练轮数。
  - `dataset-uri`：训练数据的云存储位置。
  - `model-dir`：保存训练后模型文件的云存储位置。
- 训练函数
  - `get_data()`：
      - 获取训练数据。
      - 创建输入数据集工件。
      - 将数据集工件附加为执行环境的输入。
  - `get_model()`：
      - 获取模型架构。
  - `train_model()`：
      - 训练模型。
  - `save_model()`：
      - 保存模型。
      - 创建输出模型工件。
      - 将模型工件附加为执行环境的输出。
- 初始化实验（`init()`）并在实验中启动一个运行（`start_run()`）。
- 用 `start_execution()` 包装训练过程。
- 将实验参数的任务血统记录到日志中（`log_metrics({"lineage"...)`）。
- 结束实验运行（`end_run()`）。

In [ ]:
%%writefile custom/trainer/task.py

import argparse
import os

import google.cloud.aiplatform as aiplatform

parser = argparse.ArgumentParser()
# Args for experiment
parser.add_argument('--experiment', dest='experiment',
                    required=True, type=str,
                    help='Name of experiment')
parser.add_argument('--run', dest='run',
                    required=True, type=str,
                    help='Name of run within the experiment')

# Hyperparameters for experiment
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')

parser.add_argument('--dataset-uri', dest='dataset_uri',
                    required=True, type=str,
                    help='Location of the dataset')

parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str,
                    help='Storage location for the model')
args = parser.parse_args()

def get_data(dataset_uri, execution):
    # get the training data
    
    dataset_artifact = aiplatform.Artifact.create(
        schema_title="system.Dataset", display_name="example_dataset", uri=dataset_uri
    )
    
    execution.assign_input_artifacts([dataset_artifact])

    return None

def get_model():
    # get or create the model architecture
    return None

def train_model(dataset, model, epochs):
    aiplatform.log_params({"epochs": epochs})
    # train the model
    return model

def save_model(model, model_dir, execution):
    # save the model
    
    model_artifact = aiplatform.Artifact.create(
        schema_title="system.Model", display_name="example_model", uri=model_dir
    )
    execution.assign_output_artifacts([model_artifact])

# Create a run within the experiment
aiplatform.init(experiment=args.experiment)
aiplatform.start_run(args.run)

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    dataset = get_data(args.dataset_uri, execution)
    model = get_model()
    model = train_model(dataset, model, args.epochs)
    save_model(model, args.model_dir, execution)
    
    # Store the lineage link in the experiment
    aiplatform.log_metrics({"lineage": execution.get_output_artifacts()[0].lineage_console_uri})

aiplatform.end_run()

将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer.tar.gz

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建自定义训练任务，具有以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器映像。

- `python_package_gcs_uri`：Python训练包的位置，作为一个tarball。
- `python_module_name`：Python包中训练脚本的相对路径。

*注意：*没有要求参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "example"

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
)

运行自定义训练作业

接下来，通过调用方法 `run()`，运行自定义训练作业以开始训练作业，使用以下参数：

- `args`：传递给训练脚本的参数
    - `model_dir`：存储模型的云存储位置。
    - `dataset_uri`：数据集的云存储位置。
    - `epochs`：迭代次数（超参数）。
    - `experiment`：实验的名称。
    - `run`：实验中的运行名称。
- `replica_count`：VM实例的数量。
- `machine_type`：每个VM实例的机器类型。

In [ ]:
EXPERIMENT_NAME = f"example-{uuid.uuid1()}"
aiplatform.init(experiment=EXPERIMENT_NAME)

CMDARGS = [
    "--model-dir=" + BUCKET_URI,
    "--dataset-uri=gs://example/foo.csv",
    "--epochs=5",
    f"--experiment={EXPERIMENT_NAME}",
    "--run=run-1",
]

job.run(
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    service_account=SERVICE_ACCOUNT,
    sync=True,
)

获取实验结果

接下来，使用实验名称作为参数调用方法`get_experiment_df()`以将实验结果获取为pandas dataframe。

在这个例子中，您将资源URI存储为度量值`lineage`，并将其与执行运行相关联。

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

#### 可视化工件谱系

接下来，打开下面的链接来可视化工件谱系。

In [ ]:
try:
    print("Open the following link", experiment_df["metric.lineage"][0])
except Exception as e:
    print(e)

#### 删除自定义训练任务

您可以使用`delete()`方法来删除您的自定义训练任务。

In [ ]:
job.delete()

删除实验

由于实验是在Vertex AI训练中创建的，要删除实验，您可以使用`list()`方法来获取项目中的所有实验，然后根据实验名称进行筛选。

In [ ]:
experiments = aiplatform.Experiment.list()
for experiment in experiments:
    if experiment.name == EXPERIMENT_NAME:
        experiment.delete()

清理

要清理这个项目中使用的所有谷歌云资源，您可以删除用于教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}